### Generating fake 700 values to add to the hr_dataset

In [10]:
from os import name
import os
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta, date
import json

class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Updated constants
        self.MIN_AGE = 18  # Changed to include interns
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()


    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """
        Generate a random date within specified year offsets from today.
        
        Parameters:
        start_year_offset (int): Number of years before today to start range (negative number)
        end_year_offset (int): Number of years before/after today to end range
        
        Returns:
        date: A random date within the specified range
        """
        start_date = self.TODAY + timedelta(days=365 * start_year_offset)
        end_date = self.TODAY + timedelta(days=365 * end_year_offset)
        
        # Generate random date between start and end
        days_between = (end_date - start_date).days
        random_days = np.random.randint(0, max(1, days_between))
        return start_date + timedelta(days=random_days)

    def _calculate_age(self, birth_date):
        """Calculate age based on birth date"""
        today = self.TODAY
        age = today.year - birth_date.year
        # Adjust age if birthday hasn't occurred this year
        if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
            age -= 1
        return age

    def _calculate_years_experience(self, hire_date):
        """Calculate years of experience based on hire date"""
        days_employed = (self.TODAY - hire_date).days
        return round(days_employed / 365.25, 1)

    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Intern', 'Junior Software Engineer', 'Software Engineer', 'Senior Software Engineer', 'Tech Lead', 'Engineering Manager'],
                'salary_ranges': {
                    'Intern': (45000, 65000),
                    'Junior Software Engineer': (65000, 85000),
                    'Software Engineer': (85000, 130000),
                    'Senior Software Engineer': (120000, 180000),
                    'Tech Lead': (150000, 200000),
                    'Engineering Manager': (170000, 250000)
                },
                'intern_ratio': 0.15  # 15% interns
            },
            'IT': {
                'roles': ['Intern', 'Junior System Administrator', 'System Administrator', 'Senior System Administrator', 'Network Engineer', 'Security Analyst', 'IT Manager'],
                'salary_ranges': {
                    'Intern': (40000, 55000),
                    'Junior System Administrator': (55000, 75000),
                    'System Administrator': (70000, 115000),
                    'Senior System Administrator': (90000, 140000),
                    'Network Engineer': (85000, 145000),
                    'Security Analyst': (95000, 160000),
                    'IT Manager': (130000, 190000)
                },
                'intern_ratio': 0.10
            },
            'Sales': {
                'roles': ['Sales Intern', 'Junior Sales Representative', 'Sales Representative', 'Senior Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Intern': (35000, 45000),
                    'Junior Sales Representative': (40000, 60000),
                    'Sales Representative': (50000, 90000),
                    'Senior Sales Representative': (70000, 120000),
                    'Account Executive': (85000, 150000),
                    'Sales Manager': (110000, 180000)
                },
                'intern_ratio': 0.08
            },
            'HR': {
                'roles': ['HR Intern', 'HR Assistant', 'HR Specialist', 'Senior HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Intern': (35000, 45000),
                    'HR Assistant': (40000, 55000),
                    'HR Specialist': (55000, 85000),
                    'Senior HR Specialist': (75000, 110000),
                    'Recruiter': (60000, 100000),
                    'HR Manager': (95000, 150000)
                },
                'intern_ratio': 0.05
            }
        }
        
        # Updated performance ratings with more nuance
        self.performance_ratings = {
            'Outstanding': 0.05,      # Top performers
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Inconsistent': 0.15,     # Sometimes good, sometimes bad
            'Needs Improvement': 0.10,
            'Performance Plan': 0.05
        }
        
        # Expanded recruitment sources
        self.recruitment_sources = {
            'LinkedIn': 0.25,
            'Employee Referral': 0.20,
            'Company Website': 0.15,
            'Indeed': 0.12,
            'University Partnership': 0.10,
            'Recruitment Agency': 0.08,
            'Diversity Job Fair': 0.05,
            'Stack Overflow': 0.03,
            'GitHub': 0.02
        }
        
        # More detailed termination reasons
        self.termination_reasons = {
            'N/A-StillEmployed': 0.82,
            'Voluntary-Better Opportunity': 0.05,
            'Voluntary-Career Change': 0.02,
            'Voluntary-Relocation': 0.02,
            'Voluntary-Return to School': 0.01,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Attendance': 0.01,
            'Involuntary-Policy Violation': 0.01,
            'Involuntary-Layoff': 0.02
        }
        
        # Education levels with probability distribution
        self.education_levels = {
            'High School': 0.10,
            'Some College': 0.15,
            'Associates Degree': 0.10,
            'Bachelors Degree': 0.45,
            'Masters Degree': 0.18,
            'PhD': 0.02
        }
        
        # Remote work status
        self.remote_status = {
            'On-site': 0.40,
            'Hybrid-2 Days': 0.20,
            'Hybrid-3 Days': 0.25,
            'Full Remote': 0.15
        }
        
        # Project completion rate (normalized distribution around 85%)
        self.project_completion_mean = 0.85
        self.project_completion_std = 0.15
        
        # Previous companies count distribution
        self.prev_companies_probs = {
            0: 0.20,  # Fresh graduates/first job
            1: 0.25,
            2: 0.25,
            3: 0.15,
            4: 0.10,
            5: 0.05
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }
    
    
    def _generate_employee(self):
        """Generate a single employee record with enhanced realism"""
        # Select department and role with intern consideration
        department = np.random.choice(list(self.departments.keys()))
        
        # Determine if this will be an intern based on department's intern ratio
        is_intern = np.random.random() < self.departments[department]['intern_ratio']
        
        # Filter roles based on intern status
        available_roles = [role for role in self.departments[department]['roles'] 
                         if ('intern' in role.lower()) == is_intern]
        role = np.random.choice(available_roles)
        
        # Generate dates with consideration for role
        if 'intern' in role.lower():
            hire_date = self._generate_date(start_year_offset=-1)  # Interns typically within last year
            birth_date = self._generate_date(
                start_year_offset=-(self.MIN_AGE + 5),
                end_year_offset=-(self.MIN_AGE)
            )
        else:
            hire_date = self._generate_date()
            birth_date = self._generate_date(
                start_year_offset=-(self.MAX_AGE),
                end_year_offset=-(22)  # Most non-interns are at least 22
            )
        
        # Calculate age and experience
        age = self._calculate_age(birth_date)
        years_experience = self._calculate_years_experience(hire_date)
        
        # Generate performance rating (interns more likely to be "Meets Expectations")
        if is_intern:
            performance_probs = {'Meets Expectations': 0.7, 'Exceeds Expectations': 0.15, 
                               'Needs Improvement': 0.15}
            performance = np.random.choice(
                list(performance_probs.keys()),
                p=list(performance_probs.values())
            )
        else:
            performance = np.random.choice(
                list(self.performance_ratings.keys()),
                p=list(self.performance_ratings.values())
            )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        # Generate education level appropriate for role
        if 'intern' in role.lower():
            education_options = ['Some College', 'Bachelors Degree']
            education_probs = [0.3, 0.7]
        else:
            education_options = list(self.education_levels.keys())
            education_probs = list(self.education_levels.values())
        
        education = np.random.choice(education_options, p=education_probs)
        
        # Calculate work satisfaction and productivity
        satisfaction = self._generate_work_satisfaction_metrics(years_experience, performance, role)
        productivity = self._generate_productivity_metrics(performance, years_experience)
        
        # Generate base employee record
        employee = {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EducationLevel": education,
            "YearsExperience": round(years_experience, 1),
            "PreviousCompanies": np.random.choice(
                list(self.prev_companies_probs.keys()),
                p=list(self.prev_companies_probs.values())
            ),
            "RemoteWorkStatus": np.random.choice(
                list(self.remote_status.keys()),
                p=list(self.remote_status.values())
            ),
            "ProjectCompletionRate": min(100, max(0, np.random.normal(
                self.project_completion_mean * 100,
                self.project_completion_std * 100
            ))),
            "WorkSatisfaction": satisfaction,
            "Productivity": productivity,
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d'),
            "OvertimeHours": int(np.random.exponential(10)) if not is_intern else int(np.random.exponential(3)),
            "TrainingHoursCompleted": int(np.random.normal(40, 15)) if not is_intern else int(np.random.normal(60, 10)),
            "TeamSize": int(np.random.normal(8, 3)),
            "ProjectsCompleted": int(np.random.normal(12, 4) * years_experience) if years_experience > 0 else 0,
            "CertificationsCount": int(np.random.exponential(2)),
        }
        
        # Add department-specific metrics
        if department == 'Sales':
            employee.update({
                "QuotaAttainment": round(min(150, max(50, np.random.normal(100, 20))), 1),
                "LeadConversionRate": round(min(100, max(0, np.random.normal(30, 10))), 1),
                "ClientRetentionRate": round(min(100, max(60, np.random.normal(85, 8))), 1),
                "DealsClosed": int(np.random.normal(15, 5) * years_experience) if years_experience > 0 else 0,
                "AverageContractValue": round(np.random.normal(50000, 20000), 2),
                "ClientSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "SalesTargetPercent": round(min(200, max(0, np.random.normal(95, 25))), 1),
                "ProspectingHours": int(np.random.normal(15, 5)),
                "RepeatBusinessPercent": round(min(100, max(0, np.random.normal(65, 15))), 1)
            })

        elif department == 'Engineering':
            # Calculate bug metrics based on experience and performance
            base_bug_rate = np.random.normal(5, 2)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/5)))  # More experience = fewer bugs
            performance_factors = {
                'Outstanding': 0.6,
                'Exceeds Expectations': 0.8,
                'Meets Expectations': 1.0,
                'Inconsistent': 1.3,
                'Needs Improvement': 1.5,
                'Performance Plan': 1.8
            }
            
            # Calculate code metrics
            code_review_scores = []
            for _ in range(3):  # Last 3 code reviews
                base_score = np.random.normal(8, 1)
                exp_modifier = min(2, years_experience/3)  # Experience improves code quality
                score = min(10, max(1, base_score + exp_modifier))
                code_review_scores.append(round(score, 1))
            
            employee.update({
                "CodeQualityScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "BugsPerProject": round(max(0, base_bug_rate * experience_factor * performance_factors[performance]), 1),
                "CodeReviewScores": code_review_scores,
                "CommitsPerWeek": int(np.random.normal(12, 4)),
                "PullRequestsOpened": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "PullRequestsAccepted": int(np.random.normal(7, 3) * years_experience) if years_experience > 0 else 0,
                "TechnicalDebtScore": round(min(10, max(1, np.random.normal(4, 2))), 1),
                "DocumentationContribution": round(min(100, max(0, np.random.normal(70, 20))), 1),
                "SystemUptime": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "OnCallIncidents": int(np.random.exponential(3)),
                "DeploymentFrequency": int(np.random.normal(5, 2)),
                "CodeCoverage": round(min(100, max(60, np.random.normal(85, 8))), 1)
            })

        elif department == 'IT':
            # Calculate incident resolution metrics based on experience
            base_resolution_time = np.random.normal(4, 1)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/3)))
            
            employee.update({
                "TicketsResolved": int(np.random.normal(120, 30) * years_experience) if years_experience > 0 else 0,
                "AverageResolutionTime": round(max(0.5, base_resolution_time * experience_factor), 1),
                "FirstCallResolutionRate": round(min(100, max(50, np.random.normal(75, 10))), 1),
                "SystemUptimeManaged": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "SecurityIncidentsHandled": int(np.random.exponential(5)),
                "CustomerSatisfactionScore": round(min(10, max(1, np.random.normal(8, 1))), 1),
                "SystemsManaged": int(np.random.normal(15, 5)),
                "PreventiveMaintenance": round(min(100, max(0, np.random.normal(80, 15))), 1),
                "NetworkPerformanceScore": round(min(100, max(70, np.random.normal(92, 5))), 1),
                "IncidentResponseTime": round(max(0.1, np.random.exponential(0.5)), 1),
                "BackupSuccessRate": round(min(100, max(90, np.random.normal(98, 2))), 1),
                "ComplianceScore": round(min(100, max(80, np.random.normal(95, 3))), 1)
            })

        elif department == 'HR':
            # Calculate recruiting effectiveness based on experience
            base_time_to_fill = np.random.normal(30, 8)
            experience_factor = max(0.7, min(1.3, 1 / (1 + years_experience/4)))
            
            employee.update({
                "TimeToFill": round(max(10, base_time_to_fill * experience_factor), 1),
                "CandidatesHired": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "RetentionRate": round(min(100, max(70, np.random.normal(85, 8))), 1),
                "EmployeeSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "TrainingProgramsManaged": int(np.random.normal(6, 2)),
                "PolicyComplianceRate": round(min(100, max(90, np.random.normal(97, 2))), 1),
                "DisputesResolved": int(np.random.exponential(5)),
                "OnboardingEffectivenessScore": round(min(100, max(60, np.random.normal(85, 10))), 1),
                "ExitInterviewsCompleted": int(np.random.normal(5, 2)),
                "BenefitsAdministrationScore": round(min(100, max(70, np.random.normal(90, 5))), 1),
                "RecruitmentCostPerHire": round(np.random.normal(4000, 1000), 2),
                "EmployeeRelationsScore": round(min(10, max(1, np.random.normal(8, 1))), 1)
            })

        # Add learning and development metrics
        certifications = []
        if np.random.random() < 0.7:  # 70% chance of having certifications
            cert_count = employee['CertificationsCount']
            dept_certs = {
                'Engineering': ['AWS Certified Developer', 'CISSP', 'PMP', 'Azure Developer', 'Google Cloud Engineer'],
                'IT': ['CompTIA A+', 'CCNA', 'CISSP', 'AWS SysOps', 'Microsoft Certified'],
                'Sales': ['Certified Sales Professional', 'Salesforce Certified', 'HubSpot Sales', 'Customer Success'],
                'HR': ['PHR', 'SHRM-CP', 'HRCI', 'Talent Management', 'DEI Certification']
            }
            available_certs = dept_certs.get(department, [])
            if available_certs and cert_count > 0:
                certifications = np.random.choice(available_certs, size=min(cert_count, len(available_certs)), replace=False).tolist()
        
        employee["Certifications"] = certifications
        
        # Add collaboration metrics
        employee.update({
            "CrossTeamProjects": int(np.random.normal(3, 1) * years_experience) if years_experience > 0 else 0,
            "PeerReviewsCompleted": int(np.random.normal(10, 3) * years_experience) if years_experience > 0 else 0,
            "MentoringHours": int(np.random.exponential(20)) if years_experience > 2 else 0,
            "TeamCollaborationScore": round(min(10, max(1, np.random.normal(7.5, 1.5))), 1)
        })
        
        return employee
    
    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, csv_path='hr_data.csv', json_path='hr_data.json'):
        """Generate and save dataset in both CSV and JSON formats"""
        df = self.generate_dataset(num_records)
        
        # Save as CSV
        df.to_csv(csv_path, index=False)
        
        # Save as JSON with proper formatting
        df.to_json(json_path, orient='records', date_format='iso')
        
        return df
    
# First create an instance of your HRDataGenerator
generator = HRDataGenerator()

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Define the full paths for the output files
csv_path = os.path.join(current_dir, 'hr_data.csv')
json_path = os.path.join(current_dir, 'hr_data.json')

# Generate and save the data
df = generator.save_dataset(num_records=1000, csv_path=csv_path, json_path=json_path)

# Verify the files were created
if os.path.exists(csv_path):
    print(f"CSV file created successfully at: {csv_path}")
    print(f"CSV file size: {os.path.getsize(csv_path)} bytes")
else:
    print("CSV file was not created!")

if os.path.exists(json_path):
    print(f"JSON file created successfully at: {json_path}")
    print(f"JSON file size: {os.path.getsize(json_path)} bytes")
else:
    print("JSON file was not created!")

# Print the first few rows of the generated data to verify
print("\nFirst few rows of the generated data:")
print(df.head(2))

Current working directory: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing


AttributeError: 'HRDataGenerator' object has no attribute '_generate_work_satisfaction_metrics'

### V2

In [ ]:
from openpyxl import load_workbook
import os
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta, date
import json

class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Updated constants
        self.MIN_AGE = 18  # Changed to include interns
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()
    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Intern', 'Junior Software Engineer', 'Software Engineer', 'Senior Software Engineer', 'Tech Lead', 'Engineering Manager'],
                'salary_ranges': {
                    'Intern': (45000, 65000),
                    'Junior Software Engineer': (65000, 85000),
                    'Software Engineer': (85000, 130000),
                    'Senior Software Engineer': (120000, 180000),
                    'Tech Lead': (150000, 200000),
                    'Engineering Manager': (170000, 250000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior Software Engineer': (0, 2),
                    'Software Engineer': (2, 5),
                    'Senior Software Engineer': (5, 10),
                    'Tech Lead': (8, 15),
                    'Engineering Manager': (10, 20)
                },
                'intern_ratio': 0.15
            },
            'IT': {
                'roles': ['Intern', 'Junior System Administrator', 'System Administrator', 'Senior System Administrator', 'Network Engineer', 'Security Analyst', 'IT Manager'],
                'salary_ranges': {
                    'Intern': (40000, 55000),
                    'Junior System Administrator': (55000, 75000),
                    'System Administrator': (70000, 115000),
                    'Senior System Administrator': (90000, 140000),
                    'Network Engineer': (85000, 145000),
                    'Security Analyst': (95000, 160000),
                    'IT Manager': (130000, 190000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior System Administrator': (0, 2),
                    'System Administrator': (2, 5),
                    'Senior System Administrator': (5, 10),
                    'Network Engineer': (3, 8),
                    'Security Analyst': (3, 8),
                    'IT Manager': (8, 15)
                },
                'intern_ratio': 0.10
            },
            'Sales': {
                'roles': ['Sales Intern', 'Junior Sales Representative', 'Sales Representative', 'Senior Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Intern': (35000, 45000),
                    'Junior Sales Representative': (40000, 60000),
                    'Sales Representative': (50000, 90000),
                    'Senior Sales Representative': (70000, 120000),
                    'Account Executive': (85000, 150000),
                    'Sales Manager': (110000, 180000)
                },
                'experience_requirements': {
                    'Sales Intern': (0, 1),
                    'Junior Sales Representative': (0, 2),
                    'Sales Representative': (1, 4),
                    'Senior Sales Representative': (4, 8),
                    'Account Executive': (5, 10),
                    'Sales Manager': (8, 15)
                },
                'intern_ratio': 0.08
            },
            'HR': {
                'roles': ['HR Intern', 'HR Assistant', 'HR Specialist', 'Senior HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Intern': (35000, 45000),
                    'HR Assistant': (40000, 55000),
                    'HR Specialist': (55000, 85000),
                    'Senior HR Specialist': (75000, 110000),
                    'Recruiter': (60000, 100000),
                    'HR Manager': (95000, 150000)
                },
                'experience_requirements': {
                    'HR Intern': (0, 1),
                    'HR Assistant': (0, 2),
                    'HR Specialist': (2, 5),
                    'Senior HR Specialist': (5, 10),
                    'Recruiter': (2, 8),
                    'HR Manager': (8, 15)
                },
                'intern_ratio': 0.05
            }
        }
        
        # Add previous companies distribution
        self.prev_companies_probs = {
            '0': 0.15,  # First job
            '1-2': 0.35,  # Most common
            '3-4': 0.30,
            '5-6': 0.15,
            '7+': 0.05   # Rare but possible
        }
        
        # Add project completion base metrics
        self.project_completion_mean = 0.85  # 85% average completion rate
        self.project_completion_std = 0.12   # Standard deviation for normal distribution
        
       # Updated performance ratings to ensure consistency
        self.performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.recruitment_sources = {
            'LinkedIn': 0.25,
            'Employee Referral': 0.20,
            'Company Website': 0.15,
            'Indeed': 0.12,
            'University Partnership': 0.10,
            'Recruitment Agency': 0.08,
            'Diversity Job Fair': 0.05,
            'Stack Overflow': 0.03,
            'GitHub': 0.02
        }
        
        self.termination_reasons = {
            'N/A-StillEmployed': 0.82,
            'Voluntary-Better Opportunity': 0.05,
            'Voluntary-Career Change': 0.02,
            'Voluntary-Relocation': 0.02,
            'Voluntary-Return to School': 0.01,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Attendance': 0.01,
            'Involuntary-Policy Violation': 0.01,
            'Involuntary-Layoff': 0.02
        }
        
        self.education_levels = {
            'High School': 0.10,
            'Some College': 0.15,
            'Associates Degree': 0.10,
            'Bachelors Degree': 0.45,
            'Masters Degree': 0.18,
            'PhD': 0.02
        }
        
        self.remote_status = {
            'On-site': 0.40,
            'Hybrid-2 Days': 0.20,
            'Hybrid-3 Days': 0.25,
            'Full Remote': 0.15
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }

        self.engagement_modifiers = {
                'salary_satisfaction': (-0.5, 0.5),
                'years_in_role': (-0.2, -0.1),
                'performance': {
                    'Outstanding': 0.8,
                    'Exceeds Expectations': 0.5,
                    'Meets Expectations': 0.0,
                    'Needs Improvement': -0.5,
                    'Performance Plan': -0.8
                }
        }
        
        self.base_performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.dept_performance_biases = {
            'Engineering': {
                'Outstanding': 1.2,
                'Exceeds Expectations': 1.1
            },
            'Sales': {
                'Outstanding': 1.3,
                'Inconsistent': 1.2
            }
        }


    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """
        Generate a random date within specified year offsets from today.
        
        Parameters:
        start_year_offset (int): Number of years before today to start range (negative number)
        end_year_offset (int): Number of years before/after today to end range
        
        Returns:
        date: A random date within the specified range
        """
        start_date = self.TODAY + timedelta(days=365 * start_year_offset)
        end_date = self.TODAY + timedelta(days=365 * end_year_offset)
        
        # Generate random date between start and end
        days_between = (end_date - start_date).days
        random_days = np.random.randint(0, max(1, days_between))
        return start_date + timedelta(days=random_days)

    def _calculate_age(self, birth_date):
        """Calculate age based on birth date"""
        today = self.TODAY
        age = today.year - birth_date.year
        # Adjust age if birthday hasn't occurred this year
        if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
            age -= 1
        return age

    def _calculate_years_experience(self, hire_date):
        """Calculate years of experience based on hire date"""
        days_employed = (self.TODAY - hire_date).days
        return round(days_employed / 365.25, 1)
        
    def _generate_work_satisfaction_metrics(self, years_experience, performance, role):
        """Generate realistic work satisfaction metrics based on various factors"""
        base_satisfaction = np.random.normal(7, 2)  # Base satisfaction out of 10
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 2,
            'Exceeds Expectations': 1,
            'Meets Expectations': 0,
            'Needs Improvement': -1.5,
            'Performance Plan': -2.5
        }
        
        # Adjust based on role level (more senior roles tend to be more satisfied)
        is_senior = any(senior_term in role.lower() for senior_term in ['senior', 'lead', 'manager'])
        role_adjustment = 1 if is_senior else 0
        
        # Adjust based on years of experience (slight burnout factor)
        experience_adjustment = -0.5 if years_experience > 5 else 0
        
        # Calculate final satisfaction score
        satisfaction = base_satisfaction + performance_adjustments[performance] + role_adjustment + experience_adjustment
        satisfaction = max(1, min(10, satisfaction))  # Clamp between 1 and 10
        
        return round(satisfaction, 1)

    def _generate_productivity_metrics(self, performance, years_experience):
        """Generate realistic productivity metrics"""
        base_productivity = np.random.normal(80, 10)  # Base productivity percentage
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 15,
            'Exceeds Expectations': 10,
            'Meets Expectations': 0,
            'Needs Improvement': -10,
            'Performance Plan': -15
        }
        
        # Experience adjustment (diminishing returns after 5 years)
        experience_adjustment = min(years_experience * 2, 10)
        
        productivity = base_productivity + performance_adjustments[performance] + experience_adjustment
        productivity = max(40, min(120, productivity))  # Clamp between 40% and 120%
        
        return round(productivity, 1)

    def _get_performance_distribution(self, department, years_experience, role):
        """Get performance distribution adjusted for department and experience"""
        base_dist = self.base_performance_ratings.copy()
        
        # Apply department-specific biases
        if department in self.dept_performance_biases:
            for rating, bias in self.dept_performance_biases[department].items():
                base_dist[rating] *= bias
        
        # Experience adjustments
        if years_experience < 1:
            base_dist['Exceeds Expectations'] *= 0.5
            base_dist['Meets+ Expectations'] *= 0.7
            base_dist['Meets Expectations'] *= 1.3
        elif years_experience > 5:
            base_dist['Exceeds Expectations'] *= 1.2
            base_dist['Needs Improvement'] *= 0.8
        
        # Role-specific adjustments
        if 'Senior' in role or 'Lead' in role:
            base_dist['Exceeds Expectations'] *= 1.3
            base_dist['Performance Plan'] *= 0.7
        elif 'Intern' in role:
            base_dist['Exceeds Expectations'] *= 0.6
            base_dist['Meets Expectations'] *= 1.2
        
        # Normalize probabilities
        total = sum(base_dist.values())
        return {k: v/total for k, v in base_dist.items()}

    def _generate_experience_based_role(self, department):
        """Generate role based on experience distribution with realistic ratios"""
        dept_roles = self.departments[department]['roles']
        experience_reqs = self.departments[department]['experience_requirements']
        
        # Weight roles to favor more junior positions and maintain realistic ratios
        weights = []
        for role in dept_roles:
            if 'Intern' in role:
                weights.append(0.08)  # 8% interns
            elif 'Junior' in role or 'Coordinator' in role:
                weights.append(0.25)  # 25% junior positions
            elif 'Senior' in role or 'Lead' in role:
                weights.append(0.17)  # 17% senior positions
            else:
                weights.append(0.50)  # 50% mid-level positions
        
        # Normalize weights
        weights = np.array(weights) / sum(weights)
        return np.random.choice(dept_roles, p=weights)

    def _generate_hire_date(self, role, department):
        """Generate hire date based on role and real-world patterns"""
        min_exp, max_exp = self.departments[department]['experience_requirements'][role]
        
        if 'Intern' in role:
            # Interns typically start in summer or winter
            month = np.random.choice([6, 7, 1])  # June, July, or January
            year = self.BASE_YEAR
            day = np.random.randint(1, 28)
            return date(year, month, day)
        
        if min_exp == 0:  # Entry level positions
            max_years_ago = 2
        else:
            max_years_ago = min_exp + np.random.randint(1, 4)  # Add variable buffer
        
        start_date = self.TODAY - timedelta(days=365 * max_years_ago)
        hire_date = self.fake.date_between(start_date=start_date, end_date=self.TODAY)
        
        # Adjust for common hiring months (more hiring in Q1 and Q3)
        month_weights = [1.2, 1.1, 1.2, 0.9, 0.8, 1.0, 1.1, 1.1, 1.2, 0.9, 0.8, 0.7]
        if np.random.random() < 0.7:  # 70% chance to adjust to preferred hiring months
            hire_date = hire_date.replace(month=np.random.choice(
                range(1, 13),
                p=np.array(month_weights) / sum(month_weights)
            ))
        
        return hire_date

    def _generate_age(self, role, department, hire_date):
        """Generate age based on role and realistic distributions"""
        min_exp, _ = self.departments[department]['experience_requirements'][role]
        years_since_hire = (self.TODAY - hire_date).days / 365.25
        
        if 'Intern' in role:
            return np.random.randint(18, 25)
        elif 'Junior' in role or 'Coordinator' in role:
            base_age = np.random.normal(25, 3)
        elif min_exp <= 2:
            base_age = np.random.normal(28, 4)
        elif min_exp <= 5:
            base_age = np.random.normal(32, 5)
        else:
            base_age = np.random.normal(38, 7)
        
        # Add some outliers
        if np.random.random() < 0.05:  # 5% chance of being an outlier
            if np.random.random() < 0.5:
                base_age += np.random.normal(10, 3)  # Older outlier
            else:
                base_age -= np.random.normal(5, 2)   # Younger outlier
        
        # Ensure age makes sense with hire date and experience
        min_possible_age = 18 + years_since_hire
        max_possible_age = 65
        
        age = max(min_possible_age, min(max_possible_age, base_age))
        return int(round(age))

    def _calculate_salary(self, role, department, performance, years_experience):
        """Calculate salary with realistic variations and market factors"""
        base_range = self.departments[department]['salary_ranges'][role]
        min_exp, max_exp = self.departments[department]['experience_requirements'][role]
        
        # Calculate base salary within range with slight random variation
        base = np.random.normal(
            (base_range[0] + base_range[1]) / 2,
            (base_range[1] - base_range[0]) / 6
        )
        
        # Ensure within range
        base = max(base_range[0], min(base_range[1], base))
        
        # Updated performance multipliers
        performance_multipliers = {
            'Outstanding': np.random.uniform(1.15, 1.20),
            'Exceeds Expectations': np.random.uniform(1.08, 1.14),
            'Meets Expectations': np.random.uniform(0.98, 1.02),
            'Needs Improvement': np.random.uniform(0.92, 0.97),
            'Performance Plan': np.random.uniform(0.85, 0.90)
        }

        exp_multiplier = 1 + (np.log1p(min(years_experience, max_exp)) * np.random.uniform(0.02, 0.04))
        
        market_adjustments = {
            'Engineering': np.random.uniform(1.05, 1.15),
            'IT': np.random.uniform(1.02, 1.10),
            'Sales': np.random.uniform(0.95, 1.05),
            'HR': np.random.uniform(0.98, 1.02)
        }
        
        # Role-specific adjustments
        role_adjustment = 1.0
        if 'Senior' in role:
            role_adjustment *= np.random.uniform(1.05, 1.15)
        elif 'Lead' in role:
            role_adjustment *= np.random.uniform(1.10, 1.20)
        elif 'Intern' in role:
            role_adjustment *= np.random.uniform(0.85, 0.95)
        
        # Calculate final salary with all factors
        final_salary = (base 
                    * performance_multipliers[performance]
                    * exp_multiplier
                    * market_adjustments[department]
                    * role_adjustment)
        
        # Add some random noise (±2%)
        noise = np.random.uniform(0.98, 1.02)
        final_salary *= noise
        
        return round(final_salary, 2)

    def _calculate_engagement_score(self, salary, years_experience, performance, role, termination_reason):
        """Calculate employee engagement score based on multiple factors"""
        base_score = np.random.normal(3.5, 0.5)
        
        # Salary satisfaction impact
        salary_mod = np.random.uniform(*self.engagement_modifiers['salary_satisfaction'])
        base_score += salary_mod
        
        # Years in role impact (stagnation effect)
        if years_experience > 2 and 'Senior' not in role and 'Lead' not in role:
            stagnation = np.random.uniform(*self.engagement_modifiers['years_in_role'])
            base_score += stagnation * (years_experience - 2)
        
        # Performance impact
        base_score += self.engagement_modifiers['performance'][performance]
        
        # Role-specific adjustments
        if 'Intern' in role:
            base_score += np.random.uniform(0.2, 0.5)  # Interns tend to be more engaged
        elif 'Senior' in role or 'Lead' in role:
            base_score += np.random.uniform(-0.2, 0.4)  # More variable engagement
        
        # Termination impact
        if termination_reason != 'N/A-StillEmployed':
            if 'Voluntary' in termination_reason:
                base_score -= np.random.uniform(0.5, 1.5)
            elif 'Involuntary' in termination_reason:
                base_score -= np.random.uniform(1.0, 2.0)
        
        # Ensure score is within realistic bounds (1-5)
        return round(max(1.0, min(5.0, base_score)), 2)

    def _generate_absence_days(self, performance, years_experience, role):
        """Generate realistic absence days based on multiple factors"""
        if 'Intern' in role:
            # Interns have fewer absence days due to shorter tenure
            base_days = np.random.negative_binomial(2, 0.5)
        else:
            # Base absence days following negative binomial distribution
            base_days = np.random.negative_binomial(5, 0.7)
        
        # Modify based on performance
        performance_multipliers = {
            'Exceeds Expectations': 0.7,
            'Meets+ Expectations': 0.85,
            'Meets Expectations': 1.0,
            'Needs Improvement': 1.3,
            'Performance Plan': 1.5
        }
        
        # Modify based on experience (slightly more absence with more experience)
        exp_modifier = 1 + (years_experience * 0.02)
        
        # Calculate final absence days
        absence_days = int(base_days * performance_multipliers[performance] * exp_modifier)
        
        # Add occasional outliers
        if np.random.random() < 0.05:  # 5% chance of unusual absence patterns
            absence_days += np.random.randint(10, 30)
        
        return absence_days

    def _generate_employee(self):
        """Generate a single employee record with enhanced realism"""
        # Select department and role with intern consideration
        department = np.random.choice(list(self.departments.keys()))
        
        # Determine if this will be an intern based on department's intern ratio
        is_intern = np.random.random() < self.departments[department]['intern_ratio']
        
        # Filter roles based on intern status
        available_roles = [role for role in self.departments[department]['roles'] 
                         if ('intern' in role.lower()) == is_intern]
        role = np.random.choice(available_roles)
        
        # Generate dates with consideration for role
        if 'intern' in role.lower():
            hire_date = self._generate_date(start_year_offset=-1)  # Interns typically within last year
            birth_date = self._generate_date(
                start_year_offset=-(self.MIN_AGE + 5),
                end_year_offset=-(self.MIN_AGE)
            )
        else:
            hire_date = self._generate_date()
            birth_date = self._generate_date(
                start_year_offset=-(self.MAX_AGE),
                end_year_offset=-(22)  # Most non-interns are at least 22
            )
        
        # Calculate age and experience
        age = self._calculate_age(birth_date)
        years_experience = self._calculate_years_experience(hire_date)
        
        # Generate performance rating (interns more likely to be "Meets Expectations")
        if is_intern:
            performance_probs = {'Meets Expectations': 0.7, 'Exceeds Expectations': 0.15, 
                               'Needs Improvement': 0.15}
            performance = np.random.choice(
                list(performance_probs.keys()),
                p=list(performance_probs.values())
            )
        else:
            performance = np.random.choice(
                list(self.performance_ratings.keys()),
                p=list(self.performance_ratings.values())
            )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        # Generate education level appropriate for role
        if 'intern' in role.lower():
            education_options = ['Some College', 'Bachelors Degree']
            education_probs = [0.3, 0.7]
        else:
            education_options = list(self.education_levels.keys())
            education_probs = list(self.education_levels.values())
        
        education = np.random.choice(education_options, p=education_probs)
        
        # Calculate work satisfaction and productivity
        satisfaction = self._generate_work_satisfaction_metrics(years_experience, performance, role)
        productivity = self._generate_productivity_metrics(performance, years_experience)
        
        # Generate base employee record
        employee = {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EducationLevel": education,
            "YearsExperience": round(years_experience, 1),
            "PreviousCompanies": np.random.choice(
                list(self.prev_companies_probs.keys()),
                p=list(self.prev_companies_probs.values())
            ),
            "RemoteWorkStatus": np.random.choice(
                list(self.remote_status.keys()),
                p=list(self.remote_status.values())
            ),
            "ProjectCompletionRate": min(100, max(0, np.random.normal(
                self.project_completion_mean * 100,
                self.project_completion_std * 100
            ))),
            "WorkSatisfaction": satisfaction,
            "Productivity": productivity,
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d'),
            "OvertimeHours": int(np.random.exponential(10)) if not is_intern else int(np.random.exponential(3)),
            "TrainingHoursCompleted": int(np.random.normal(40, 15)) if not is_intern else int(np.random.normal(60, 10)),
            "TeamSize": int(np.random.normal(8, 3)),
            "ProjectsCompleted": int(np.random.normal(12, 4) * years_experience) if years_experience > 0 else 0,
            "CertificationsCount": int(np.random.exponential(2)),
        }
        
        # Add department-specific metrics
        if department == 'Sales':
            employee.update({
                "QuotaAttainment": round(min(150, max(50, np.random.normal(100, 20))), 1),
                "LeadConversionRate": round(min(100, max(0, np.random.normal(30, 10))), 1),
                "ClientRetentionRate": round(min(100, max(60, np.random.normal(85, 8))), 1),
                "DealsClosed": int(np.random.normal(15, 5) * years_experience) if years_experience > 0 else 0,
                "AverageContractValue": round(np.random.normal(50000, 20000), 2),
                "ClientSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "SalesTargetPercent": round(min(200, max(0, np.random.normal(95, 25))), 1),
                "ProspectingHours": int(np.random.normal(15, 5)),
                "RepeatBusinessPercent": round(min(100, max(0, np.random.normal(65, 15))), 1)
            })

        elif department == 'Engineering':
            # Calculate bug metrics based on experience and performance
            base_bug_rate = np.random.normal(5, 2)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/5)))  # More experience = fewer bugs
            performance_factors = {
                'Outstanding': 0.6,
                'Exceeds Expectations': 0.8,
                'Meets Expectations': 1.0,
                'Inconsistent': 1.3,
                'Needs Improvement': 1.5,
                'Performance Plan': 1.8
            }
            
            # Calculate code metrics
            code_review_scores = []
            for _ in range(3):  # Last 3 code reviews
                base_score = np.random.normal(8, 1)
                exp_modifier = min(2, years_experience/3)  # Experience improves code quality
                score = min(10, max(1, base_score + exp_modifier))
                code_review_scores.append(round(score, 1))
            
            employee.update({
                "CodeQualityScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "BugsPerProject": round(max(0, base_bug_rate * experience_factor * performance_factors[performance]), 1),
                "CodeReviewScores": code_review_scores,
                "CommitsPerWeek": int(np.random.normal(12, 4)),
                "PullRequestsOpened": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "PullRequestsAccepted": int(np.random.normal(7, 3) * years_experience) if years_experience > 0 else 0,
                "TechnicalDebtScore": round(min(10, max(1, np.random.normal(4, 2))), 1),
                "DocumentationContribution": round(min(100, max(0, np.random.normal(70, 20))), 1),
                "SystemUptime": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "OnCallIncidents": int(np.random.exponential(3)),
                "DeploymentFrequency": int(np.random.normal(5, 2)),
                "CodeCoverage": round(min(100, max(60, np.random.normal(85, 8))), 1)
            })

        elif department == 'IT':
            # Calculate incident resolution metrics based on experience
            base_resolution_time = np.random.normal(4, 1)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/3)))
            
            employee.update({
                "TicketsResolved": int(np.random.normal(120, 30) * years_experience) if years_experience > 0 else 0,
                "AverageResolutionTime": round(max(0.5, base_resolution_time * experience_factor), 1),
                "FirstCallResolutionRate": round(min(100, max(50, np.random.normal(75, 10))), 1),
                "SystemUptimeManaged": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "SecurityIncidentsHandled": int(np.random.exponential(5)),
                "CustomerSatisfactionScore": round(min(10, max(1, np.random.normal(8, 1))), 1),
                "SystemsManaged": int(np.random.normal(15, 5)),
                "PreventiveMaintenance": round(min(100, max(0, np.random.normal(80, 15))), 1),
                "NetworkPerformanceScore": round(min(100, max(70, np.random.normal(92, 5))), 1),
                "IncidentResponseTime": round(max(0.1, np.random.exponential(0.5)), 1),
                "BackupSuccessRate": round(min(100, max(90, np.random.normal(98, 2))), 1),
                "ComplianceScore": round(min(100, max(80, np.random.normal(95, 3))), 1)
            })

        elif department == 'HR':
            # Calculate recruiting effectiveness based on experience
            base_time_to_fill = np.random.normal(30, 8)
            experience_factor = max(0.7, min(1.3, 1 / (1 + years_experience/4)))
            
            employee.update({
                "TimeToFill": round(max(10, base_time_to_fill * experience_factor), 1),
                "CandidatesHired": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "RetentionRate": round(min(100, max(70, np.random.normal(85, 8))), 1),
                "EmployeeSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "TrainingProgramsManaged": int(np.random.normal(6, 2)),
                "PolicyComplianceRate": round(min(100, max(90, np.random.normal(97, 2))), 1),
                "DisputesResolved": int(np.random.exponential(5)),
                "OnboardingEffectivenessScore": round(min(100, max(60, np.random.normal(85, 10))), 1),
                "ExitInterviewsCompleted": int(np.random.normal(5, 2)),
                "BenefitsAdministrationScore": round(min(100, max(70, np.random.normal(90, 5))), 1),
                "RecruitmentCostPerHire": round(np.random.normal(4000, 1000), 2),
                "EmployeeRelationsScore": round(min(10, max(1, np.random.normal(8, 1))), 1)
            })

        # Add learning and development metrics
        certifications = []
        if np.random.random() < 0.7:  # 70% chance of having certifications
            cert_count = employee['CertificationsCount']
            dept_certs = {
                'Engineering': ['AWS Certified Developer', 'CISSP', 'PMP', 'Azure Developer', 'Google Cloud Engineer'],
                'IT': ['CompTIA A+', 'CCNA', 'CISSP', 'AWS SysOps', 'Microsoft Certified'],
                'Sales': ['Certified Sales Professional', 'Salesforce Certified', 'HubSpot Sales', 'Customer Success'],
                'HR': ['PHR', 'SHRM-CP', 'HRCI', 'Talent Management', 'DEI Certification']
            }
            available_certs = dept_certs.get(department, [])
            if available_certs and cert_count > 0:
                certifications = np.random.choice(available_certs, size=min(cert_count, len(available_certs)), replace=False).tolist()
        
        employee["Certifications"] = certifications
        
        # Add collaboration metrics
        employee.update({
            "CrossTeamProjects": int(np.random.normal(3, 1) * years_experience) if years_experience > 0 else 0,
            "PeerReviewsCompleted": int(np.random.normal(10, 3) * years_experience) if years_experience > 0 else 0,
            "MentoringHours": int(np.random.exponential(20)) if years_experience > 2 else 0,
            "TeamCollaborationScore": round(min(10, max(1, np.random.normal(7.5, 1.5))), 1)
        })
        
        return employee

    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, xlsx_path='hr_data.xlsx', json_path='hr_data.json'):
        """Generate and save dataset in both Excel and JSON formats with auto-adjusted column widths"""
        df = self.generate_dataset(num_records)
        
        # Save the DataFrame to Excel first
        df.to_excel(xlsx_path, index=False)
        
        # Adjust column widths
        workbook = load_workbook(xlsx_path)
        worksheet = workbook.active
        
        for column_cells in worksheet.columns:
            # Get the length of the longest cell in the column (including header)
            max_length = max(len(str(cell.value)) for cell in column_cells)
            column_letter = column_cells[0].column_letter  # Get column letter
            worksheet.column_dimensions[column_letter].width = max_length + 2  # Add padding
        
        # Save the adjusted workbook
        workbook.save(xlsx_path)
        
        # Also save JSON as before
        df.to_json(json_path, orient='records', date_format='iso')
        
        return df


# First create an instance of your HRDataGenerator
generator = HRDataGenerator()

# Get the current working directory
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Define the full paths for the output files
xlsx_path = os.path.join(current_dir, 'hr_data.xlsx')
json_path = os.path.join(current_dir, 'hr_data.json')

# Generate and save the data
df = generator.save_dataset(num_records=1530, xlsx_path=xlsx_path, json_path=json_path)

# Verify the files were created
if os.path.exists(xlsx_path):
    print(f"CSV file created successfully at: {xlsx_path}")
    print(f"CSV file size: {os.path.getsize(xlsx_path)} bytes")
else:
    print("CSV file was not created!")

if os.path.exists(json_path):
    print(f"JSON file created successfully at: {json_path}")
    print(f"JSON file size: {os.path.getsize(json_path)} bytes")
else:
    print("JSON file was not created!")

# Print the first few rows of the generated data to verify
print("\nFirst few rows of the generated data:")
print(df.head(2))

Current working directory: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing
CSV file created successfully at: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing\hr_data.xlsx
CSV file size: 256223 bytes
JSON file created successfully at: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing\hr_data.json
JSON file size: 1879468 bytes

First few rows of the generated data:
   EmployeeID             Name Department                     Position  \
0       93810  Patrick Sanchez      Sales  Senior Sales Representative   
1       28289    Megan Mcclain         HR                   HR Manager   

     HireDate TerminationDate  TerminationReason EmploymentStatus DateOfBirth  \
0  2017-11-03            None  N/A-StillEmployed           Active  1992-08-20   
1  2023-05-13            None  N/A-StillEmployed           Active  1974-04-29   

   Age  ... CodeReviewScores CommitsPerWeek  PullRequestsOpened  \
0   32  ...              NaN            NaN                 NaN 

### V1

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta, date
import json

class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Constants that affect data generation
        self.MIN_AGE = 22
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()
    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Software Engineer', 'Senior Software Engineer', 'QA Engineer'],
                'salary_ranges': {
                    'Software Engineer': (75000, 120000),
                    'Senior Software Engineer': (110000, 180000),
                    'QA Engineer': (65000, 110000)
                }
            },
            'IT': {
                'roles': ['System Administrator', 'Network Engineer', 'Security Analyst'],
                'salary_ranges': {
                    'System Administrator': (65000, 115000),
                    'Network Engineer': (80000, 140000),
                    'Security Analyst': (90000, 150000)
                }
            },
            'Sales': {
                'roles': ['Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Representative': (45000, 90000),
                    'Account Executive': (70000, 130000),
                    'Sales Manager': (100000, 160000)
                }
            },
            'HR': {
                'roles': ['HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Specialist': (50000, 85000),
                    'Recruiter': (55000, 95000),
                    'HR Manager': (90000, 140000)
                }
            }
        }
        
        self.performance_ratings = {
            'Exceeds Expectations': 0.15,  # 15% of employees
            'Meets Expectations': 0.65,    # 65% of employees
            'Needs Improvement': 0.15,     # 15% of employees
            'Performance Plan': 0.05       # 5% of employees
        }
        
        self.recruitment_sources = {
            'LinkedIn': 0.30,
            'Employee Referral': 0.25,
            'Company Website': 0.20,
            'Indeed': 0.15,
            'Recruitment Agency': 0.05,
            'Diversity Job Fair': 0.05
        }
        
        self.termination_reasons = {
            'N/A-StillEmployed': 0.85,    # 85% still employed
            'Voluntary-Other Opportunity': 0.06,
            'Voluntary-Relocation': 0.03,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Layoff': 0.02
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }
    
    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """Generate a date between start and end year offsets from current year"""
        start_date = date(self.BASE_YEAR + start_year_offset, 1, 1)
        end_date = date(self.BASE_YEAR + end_year_offset, 12, 31)
        fake_date = self.fake.date_between(start_date=start_date, end_date=end_date)
        return fake_date
    
    def _calculate_salary(self, role, department, performance, years_experience):
        """Calculate salary based on role, performance, and experience"""
        base_range = self.departments[department]['salary_ranges'][role]
        
        # Calculate base salary within range
        base = np.random.uniform(base_range[0], base_range[1])
        
        # Apply performance multiplier
        performance_multipliers = {
            'Exceeds Expectations': 1.15,
            'Meets Expectations': 1.0,
            'Needs Improvement': 0.9,
            'Performance Plan': 0.85
        }
        
        # Apply experience multiplier (1-3% increase per year)
        experience_multiplier = 1 + (years_experience * np.random.uniform(0.01, 0.03))
        
        final_salary = base * performance_multipliers[performance] * experience_multiplier
        return round(final_salary, 2)
    
    def _calculate_age(self, birth_date):
        """Calculate age from birth date"""
        today = self.TODAY
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    
    def _calculate_years_experience(self, hire_date):
        """Calculate years of experience from hire date"""
        days_employed = (self.TODAY - hire_date).days
        return days_employed / 365.25
    
    def _generate_employee(self):
        """Generate a single employee record"""
        # Select department and role
        department = np.random.choice(list(self.departments.keys()))
        role = np.random.choice(self.departments[department]['roles'])
        
        # Generate dates
        hire_date = self._generate_date()
        birth_date = self._generate_date(
            start_year_offset=-(self.MAX_AGE),
            end_year_offset=-(self.MIN_AGE)
        )
        
        # Calculate age and experience
        age = self._calculate_age(birth_date)
        years_experience = self._calculate_years_experience(hire_date)
        
        # Generate performance rating
        performance = np.random.choice(
            list(self.performance_ratings.keys()),
            p=list(self.performance_ratings.values())
        )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date (within the past year, before termination if applicable)
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        return {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d')
        }
    
    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, csv_path='hr_data.csv', json_path='hr_data.json'):
        """Generate and save dataset in both CSV and JSON formats"""
        df = self.generate_dataset(num_records)
        
        # Save as CSV
        df.to_csv(csv_path, index=False)
        
        # Save as JSON with proper formatting
        df.to_json(json_path, orient='records', date_format='iso')
        
        return df

# Usage example
if __name__ == "__main__":
    generator = HRDataGenerator(seed=42)
    df = generator.save_dataset(1000)
    print(f"Generated {len(df)} employee records")

Generated 1000 employee records


In [6]:
# Create generator instance
generator = HRDataGenerator(seed=42)

# Generate and save dataset
df = generator.save_dataset(
    num_records=720,
    csv_path='hr_data.csv',
    json_path='hr_data.json'
)

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'datetime.date'

### Converting it into desired format

import json
from datetime import datetime
import os
from pathlib import Path

def transform_json(input_json):
    """
    Transform JSON from current format to desired format, adding missing fields
    with default values where necessary.
    """
    # Create default values for missing fields
    default_values = {
        "ManagerName": None,
        "ManagerID": None,
        "RecruitmentSource": None,
        "PerformanceScore": None,
        "EngagementSurvey": None,
        "EmpSatisfaction": None,
        "SpecialProjectsCount": 0,
        "LastPerformanceReview_Date": None,
        "DaysLateLast30": 0,
        "Absences": 0
    }
    
    # Transform date formats
    def transform_date(date_str):
        if not date_str:
            return None
        try:
            # Parse the input date
            if "/" in date_str:
                date_obj = datetime.strptime(date_str.strip(), '%m/%d/%Y')
            else:
                return date_str  # Return as-is if format is unknown
            # Format to desired output
            return date_obj.strftime('%-m/%-d/%y')
        except ValueError:
            return date_str  # Return original if parsing fails
    
    # Transform existing fields and add new ones
    transformed = {
        # Existing fields with transformations
        "Employee_Name": f"{input_json['Employee_Name'].split()[-1]}, {' '.join(input_json['Employee_Name'].split()[:-1])}",
        "EmpID": input_json["EmpID"],
        "MarriedID": input_json["MarriedID"],
        "MaritalStatusID": input_json["MaritalStatusID"],
        "GenderID": input_json["GenderID"],
        "EmpStatusID": input_json["EmpStatusID"],
        "DeptID": input_json["DeptID"],
        "PerfScoreID": input_json["PerfScoreID"],
        "FromDiversityJobFairID": input_json["FromDiversityJobFairID"],
        "Salary": input_json["Salary"],
        "Termd": input_json["Termd"],
        "PositionID": input_json["PositionID"],
        "Position": input_json["Position"],
        "State": input_json["State"],
        "Zip": input_json["Zip"],
        "DOB": transform_date(input_json["DOB"]),
        "Sex": input_json["Sex"] + " " if len(input_json["Sex"]) == 1 else input_json["Sex"],
        "MaritalDesc": input_json["MaritalDesc"],
        "CitizenDesc": input_json["CitizenDesc"],
        "HispanicLatino": input_json["HispanicLatino"],
        "RaceDesc": input_json["RaceDesc"],
        "DateofHire": transform_date(input_json["DateofHire"]),
        "DateofTermination": transform_date(input_json["DateofTermination"]) if input_json["DateofTermination"] != "N/A-StillEmployed" else None,
        "TermReason": input_json["TermReason"],
        "EmploymentStatus": input_json["EmploymentStatus"],
        "Department": input_json["Department"].ljust(15)
    }
    
    # Add missing fields with default values
    transformed.update(default_values)
    
    return transformed

def process_json_file(input_path, output_path):
    """
    Process a JSON file from input path and save the transformed result to output path.
    
    Args:
        input_path (str): Path to the input JSON file
        output_path (str): Path where the transformed JSON should be saved
    
    Returns:
        bool: True if successful, False if an error occurred
    """
    try:
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(output_path)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        # Read input file
        with open(input_path, 'r', encoding='utf-8') as file:
            input_data = json.load(file)
        
        # Handle both single object and list of objects
        if isinstance(input_data, list):
            transformed_data = [transform_json(item) for item in input_data]
        else:
            transformed_data = transform_json(input_data)
        
        # Write transformed data to output file
        with open(output_path, 'w', encoding='utf-8') as file:
            json.dump(transformed_data, file, indent=4)
            
        print(f"Successfully transformed JSON data from {input_path}")
        print(f"Transformed data saved to {output_path}")
        return True
        
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_path}")
        return False
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in input file {input_path}")
        return False
    except Exception as e:
        print(f"Error occurred while processing the file: {str(e)}")
        return False


if __name__ == "__main__":
    input_file_path = "C:\\Users\\govar\\OneDrive\\Documents\\HRM\\data\\processed\\expanded_hr_dataset.json"
    output_file_path = "C:\\Users\\govar\\OneDrive\\Documents\\HRM\\data\\processed\\transformed.json"
    
    # Process the file
    success = process_json_file(input_file_path, output_file_path)
    
    if success:
        print("JSON transformation completed successfully!")
    else:
        print("JSON transformation failed. Please check the error messages above.")